Импортируем необходимые библиотеки

In [1]:
import os
import pandas as pd
import torch
from torch import nn
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

Зададим основные параметры

In [2]:
batch_size = 32 * 2
num_classes = 10
epochs = 10
learning_rate = 0.0001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

При необходимости загружаем датасет, разбиваем на тренировочную и валидационную выборки

In [3]:
need_download = not os.path.exists('data')
train = torchvision.datasets.CIFAR10(root = 'data', train = True, download = need_download, transform = ToTensor())
dataset = DataLoader(train, batch_size = batch_size, shuffle = True)
test = torchvision.datasets.CIFAR10(root = 'data', train = False, download = need_download, transform = ToTensor())
dataset_test = DataLoader(test, batch_size = batch_size, shuffle = True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


Функция для тренировки модели

In [4]:
def basic_training(model, allow_print=True) -> nn.Module:
    if allow_print:
        print(f"{model.__class__.__name__} is training")
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr = learning_rate)
    loss_fn = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        model.train()
        for batch in dataset:
            data, target = batch
            data, target = data.to(device), target.to(device)
            opt.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            opt.step()
        if allow_print:
            print(f"Epoch: {epoch}, Loss: {loss.item()}")
    if allow_print:
        print("Training done")
    return model

Функция для вычисления accuracy на случайных данных из датасета

In [7]:
def test_model(model):
    model.to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in dataset_test:
            data, target = batch
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    print(f"Accuracy: {100 * correct / total}% {correct}/{total}")

Модель только на полносвязных слоях

In [5]:
class DenseModel(torch.nn.Module):
    def __init__(self):
        super(DenseModel, self).__init__()
        IMG_SIZE = 32 * 32 * 3
        self.fc1 = nn.Linear(IMG_SIZE, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Пробуем обучить и оценить качество модели

In [8]:
model = DenseModel()
model = basic_training(model)
test_model(model)

DenseModel is training
Epoch: 0, Loss: 1.698836326599121
Epoch: 1, Loss: 1.9554426670074463
Epoch: 2, Loss: 1.5239547491073608
Epoch: 3, Loss: 1.627908706665039
Epoch: 4, Loss: 1.90757417678833
Epoch: 5, Loss: 2.1757261753082275
Epoch: 6, Loss: 1.5683376789093018
Epoch: 7, Loss: 1.560636043548584
Epoch: 8, Loss: 1.7746939659118652
Epoch: 9, Loss: 1.6102919578552246
Training done
Accuracy: 45.06% 4506/10000


Модель со сверточными слоями

In [9]:
class ModelConvolutionalLayers(torch.nn.Module):
    def __init__(self):
        super(ModelConvolutionalLayers, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        )
        self.fc = nn.Linear(4 * 4 * 128, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

Пробуем обучить и оценить качество модели

In [10]:
model = ModelConvolutionalLayers()
basic_training(model)
test_model(model)

ModelConvolutionalLayers is training
Epoch: 0, Loss: 1.4111193418502808
Epoch: 1, Loss: 1.316953182220459
Epoch: 2, Loss: 1.665270209312439
Epoch: 3, Loss: 1.06101655960083
Epoch: 4, Loss: 1.1594396829605103
Epoch: 5, Loss: 1.3241795301437378
Epoch: 6, Loss: 1.4407851696014404
Epoch: 7, Loss: 1.011759877204895
Epoch: 8, Loss: 0.8370255827903748
Epoch: 9, Loss: 1.484830379486084
Training done
Accuracy: 61.92% 6192/10000


Сравниваем модели с различным количеством слоев и нейронов в слоях

In [11]:
class ModelSmallAmountLayers(torch.nn.Module):
    def __init__(self):
        super(ModelSmallAmountLayers, self).__init__()
        self.fc = nn.Linear(32 * 32 * 3, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

Пробуем обучить и оценить качество модели

In [12]:
model = ModelSmallAmountLayers()
basic_training(model)
test_model(model)

ModelSmallAmountLayers is training
Epoch: 0, Loss: 1.9287559986114502
Epoch: 1, Loss: 2.1131954193115234
Epoch: 2, Loss: 1.4724221229553223
Epoch: 3, Loss: 1.7251276969909668
Epoch: 4, Loss: 1.6508307456970215
Epoch: 5, Loss: 1.52796471118927
Epoch: 6, Loss: 1.6328288316726685
Epoch: 7, Loss: 1.9062355756759644
Epoch: 8, Loss: 1.7666786909103394
Epoch: 9, Loss: 1.8646047115325928
Training done
Accuracy: 39.53% 3953/10000


Можно сделать вывод о том, что модель с всего лишь одним слоем показывает существенно худшие результаты в сравнении с, к примеру, моделью со сверточными слоями. В общем случае, чем более многослойна модель и чем больше нейронов содержится в каждом ее слое, тем более сложные -- с точки зрения разделимости -- объекты она способна классифицировать, однако в месте с тем ростут и требования к ресурсам

Дропаут

In [14]:
class ModelDropout(torch.nn.Module):
    def __init__(self):
        super(ModelDropout, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Dropout(0.2),
            nn.Flatten(),
            nn.Linear(4 * 4 * 128, 128),
            nn.ReLU(),
        )
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.model(x)
        return x

Пробуем обучить и оценить качество модели

In [15]:
epochs = 20
model = ModelDropout()
model = basic_training(model)
test_model(model)
epochs = 10

ModelDropout is training
Epoch: 0, Loss: 1.7836321592330933
Epoch: 1, Loss: 2.6240365505218506
Epoch: 2, Loss: 2.274258613586426
Epoch: 3, Loss: 2.252229928970337
Epoch: 4, Loss: 2.1617302894592285
Epoch: 5, Loss: 1.759207010269165
Epoch: 6, Loss: 1.2913119792938232
Epoch: 7, Loss: 1.5958476066589355
Epoch: 8, Loss: 2.24477219581604
Epoch: 9, Loss: 2.0385656356811523
Epoch: 10, Loss: 1.0530322790145874
Epoch: 11, Loss: 2.023738384246826
Epoch: 12, Loss: 0.8147720098495483
Epoch: 13, Loss: 1.7990814447402954
Epoch: 14, Loss: 2.3878209590911865
Epoch: 15, Loss: 0.8201456665992737
Epoch: 16, Loss: 1.9319918155670166
Epoch: 17, Loss: 1.5693658590316772
Epoch: 18, Loss: 1.5849485397338867
Epoch: 19, Loss: 0.9239271283149719
Training done
Accuracy: 61.65% 6165/10000
